<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/cda-2022/blob/main/notebooks/cda_11_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the data

In [ ]:
!wget -q --show-progress -O beer-gee.xlsx https://github.com/DepartmentOfStatisticsPUE/cda-2022/blob/main/data/beer-gee.xlsx?raw=true 

## Python solution

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [18]:
df = pd.read_excel("beer-gee.xlsx")
df.head(n=10)

,id,size,locality,education,beers,time
0,1,4,50-99k,profession,8,1
1,1,4,50-99k,profession,12,2
2,1,4,50-99k,profession,13,3
3,1,4,50-99k,profession,6,4
4,1,4,50-99k,profession,6,5
5,1,4,50-99k,profession,18,6
6,1,4,50-99k,profession,8,7
7,1,4,50-99k,profession,2,8
8,1,4,50-99k,profession,1,9
9,1,4,50-99k,profession,8,10


In [23]:
m1 = sm.GEE.from_formula(
    "beers ~ size + locality + education + C(time)",  
    groups = df["id"], 
    data = df,
    cov_struct=sm.cov_struct.Independence(), 
    family=sm.families.Poisson()).fit()

m2 = sm.GEE.from_formula(
    "beers ~ size + locality + education + C(time)",  
    groups = df["id"], 
    data = df,
    cov_struct=sm.cov_struct.Exchangeable(), 
    family=sm.families.Poisson()).fit()

There is the problem with autoregressive model

In [ ]:
m3 = sm.GEE.from_formula(
    "beers ~ size + locality + education + C(time)",  
    groups = df["id"], 
    data = df,
    time = df["time"],
    cov_struct=sm.cov_struct.Autoregressive(), 
    family=sm.families.Poisson()).fit()

In [29]:
print(m1.summary())

                               GEE Regression Results                              
Dep. Variable:                       beers   No. Observations:                 5868
Model:                                 GEE   No. clusters:                      489
Method:                        Generalized   Min. cluster size:                  12
                      Estimating Equations   Max. cluster size:                  12
Family:                            Poisson   Mean cluster size:                12.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Wed, 25 May 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         07:26:49
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.1452      0.278     11.331    

In [30]:
print(m2.summary())

                               GEE Regression Results                              
Dep. Variable:                       beers   No. Observations:                 5868
Model:                                 GEE   No. clusters:                      489
Method:                        Generalized   Min. cluster size:                  12
                      Estimating Equations   Max. cluster size:                  12
Family:                            Poisson   Mean cluster size:                12.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Wed, 25 May 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         07:26:57
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   3.1574      0.278     11.354    

Estimated $\alpha$ can be found here.

In [34]:
m2.cov_struct.dep_params

0.7062544156049541